In [1]:
from captcha_utils.generator import *
from models.cnn_ctc import *


train_path = 'E:\\traindata\\captcha_origin\\train'
validate_path = 'E:\\traindata\\captcha_origin\\test'
img_w = 200
img_h = 60
channel = 1
downsample_factor = 4


model, test_func = icp_model()
# image_gen = TextImageGenerator(train_path, validate_path, img_w, img_h, channel, downsample_factor)
# 使用随机的训练集
image_gen = RandomTextImageGenerator(train_path, validate_path, img_w, img_h, channel, downsample_factor)

# 测试
# test, _ = next(image_gen.get_next_train(10))
# test, _ = next(image_gen.get_next_val(5))
# x_input = test['the_input']
# print(x_input.shape)
# print(image_gen.cur_train_idx)
# print(image_gen.cur_vald_idx)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


E:\Github\machine_learning\image\qrcode\models\cnn_ctc.py:64: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("th..., outputs=Tensor("so...)`
  base_model = Model(input=input_data, output=y_pred)


In [2]:
import keras.callbacks
import datetime

class VizCallback(keras.callbacks.Callback):

    def __init__(self, run_name, test_func, text_img_gen, output_dir, num_display_words=6):
        self.test_func = test_func
        self.output_dir = os.path.join(
            output_dir, run_name)
        self.text_img_gen = text_img_gen
        self.num_display_words = num_display_words
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)

    def on_epoch_end(self, epoch, logs={}):
        acc_ratio = 0
        if epoch % 10 == 0:  # 每10个周期计算一次正确率
            word_batch = next(self.text_img_gen)[0]
            res = decode_batch(self.test_func, TextImageGenerator.labels_to_text, word_batch['the_input'])
            acc = 0
            total = word_batch['the_input'].shape[0]
            for i in range(total):
                if word_batch['source_str'][i].lower() == res[i].lower():
                    acc += 1
            acc_ratio = 100 * acc / total
            print('正确率: %0.5f' % acc_ratio)
        if acc_ratio > 50 or epoch % 100 == 0 :
            self.model.save_weights(os.path.join(self.output_dir, 'weights%02d_acc_%0.5f.h5' % (epoch, acc_ratio)))
        word_batch = next(self.text_img_gen)[0]
        res = decode_batch(self.test_func, TextImageGenerator.labels_to_text, word_batch['the_input'][0:self.num_display_words])
        if word_batch['the_input'][0].shape[0] < 256:
            cols = 2
        else:
            cols = 1
        for i in range(self.num_display_words):
            plt.subplot(self.num_display_words // cols, cols, i + 1)
            if K.image_data_format() == 'channels_first':
                the_input = word_batch['the_input'][i, 0, :, :]
            else:
                the_input = word_batch['the_input'][i, :, :, 0]
            plt.imshow(the_input, cmap='Greys_r')
            plt.xlabel('T = \'%s\' Decoed = \'%s\'' % (word_batch['source_str'][i], res[i]))
        plt.savefig(os.path.join(self.output_dir, 'e%02d.png' % (epoch)))
        plt.close()

        
vic_gen = RandomTextImageGenerator(train_path, validate_path, img_w, img_h, channel, downsample_factor)
vic = VizCallback(datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S'), test_func, vic_gen.get_next_val(), 
                  output_dir='E:\\Github\\machine_learning\\image\\qrcode\\logs')

In [ ]:
callbacks = [vic]
train(model, image_gen.get_next_train(100), image_gen.get_next_val(200), callbacks=callbacks, epochs=501, 
      weight='E:\\Github\\machine_learning\\image\\qrcode\\logs\\2019_05_28_01_33_41\\weights500_acc_37.00000.h5')

Epoch 1/501
100/100 [==============================] - 44s 438ms/step - loss: 0.4000 - val_loss: 4.9049
正确率: 38.00000
Epoch 2/501
100/100 [==============================] - 40s 399ms/step - loss: 0.4448 - val_loss: 4.7388
Epoch 3/501
100/100 [==============================] - 41s 408ms/step - loss: 0.4506 - val_loss: 4.8401
Epoch 4/501
100/100 [==============================] - 40s 404ms/step - loss: 0.4471 - val_loss: 4.8514
Epoch 5/501
100/100 [==============================] - 41s 405ms/step - loss: 0.4499 - val_loss: 4.6089
Epoch 6/501
100/100 [==============================] - 41s 405ms/step - loss: 0.4313 - val_loss: 4.7706
Epoch 7/501
100/100 [==============================] - 41s 405ms/step - loss: 0.4487 - val_loss: 5.1239
Epoch 8/501
100/100 [==============================] - 40s 404ms/step - loss: 0.4518 - val_loss: 4.5652
Epoch 9/501
100/100 [==============================] - 41s 406ms/step - loss: 0.4329 - val_loss: 4.6086
Epoch 10/501
100/100 [============================

In [ ]:
# from captcha_utils.icp_factory import *

# test()